In [1]:
import pandas as pd
import numpy as np
import json
from pandas.api.types import is_numeric_dtype

# Load the CSV file
file_path = 'playoff_series.csv'
df = pd.read_csv(file_path)
print(df.columns)
# Rename the columns and divide PTS by 100

df = df.rename(columns={'PLAYER_ID': 'nba_id', 'PLAYER_NAME': 'player_name'})

# Select the specified columns
columns_to_keep = [
    'nba_id', 'player_name', 'year', 'round', 'OPP', 'USG_PCT', 'TS_PCT','TEAM_ID',
    'year_avg_ts', 'OPP_TS_PCT', 'rOPP_TS_PCT', 'GP', 'PTS', 'MIN', 'AGE',
    'AST_PCT', 'POSS', 'OFF_RATING', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A',
    'FG3_PCT', 'FTM', 'FTA', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK',
    'BLKA', 'PF', 'PFD', 'PLUS_MINUS'
]

df_selected = df[columns_to_keep].reset_index(drop=True)
print(df_selected[df_selected.player_name=='Stephen Curry']['PTS'])

# Create TSA column

# Function to safely divide
def safe_divide_num(a, b):
    if a==0 and b==0:
        return 0
    return np.where( (b != 0) &np.isfinite(b) & np.isfinite(a), a / b, 0)

# Function for weighted average
def safe_divide(a, b):
    result = pd.Series(0, index=a.index)
    mask = (b != 0) & np.isfinite(b) & np.isfinite(a)
    result[mask] = a[mask] / b[mask]
    return result
def weighted_average(group, value_col, weight_col):
    numerator = (group[value_col] * group[weight_col]).sum()
    denominator = group[weight_col].sum()
    

    wa = safe_divide_num(numerator, denominator)
    return wa

pct_col =[col for col in columns_to_keep if 'pct' in col.lower()]
sum_col=[col for col in columns_to_keep if is_numeric_dtype(df[col])==True]

sum_col = [col for col in sum_col if col not in ['nba_id','player_id','year','round','OPP']]
pct_col = [col for col in pct_col if col not in ['nba_id','player_id','year','round','OPP']]
sum_col=[col for col in sum_col if col not in pct_col]
id_col=['nba_id','player_name','year']


print(sum_col)
print(pct_col)
def weighted_frame(df,id_col,pct_col,sum_col):
    pct_df=[]
    for col in pct_col:
        new_columns =id_col.copy()
        new_columns.append(col)
        col_df= df.groupby(id_col).apply(weighted_average,col,'POSS').reset_index()
        col_df.columns=new_columns
        pct_df.append(col_df)

    df_grouped_pct = pct_df[0]
    for frame in pct_df[1:]:
        df_grouped_pct= df_grouped_pct.merge(frame)
    df_grouped_sum=df.groupby(id_col).sum()[sum_col].reset_index()


    df_grouped = df_grouped_pct.merge(df_grouped_sum,on=id_col)
    return df_grouped
df_grouped_year = weighted_frame(df_selected,id_col,pct_col,sum_col)

df_grouped_career = weighted_frame(df_selected,['nba_id','player_name'],pct_col,sum_col)
df_grouped_career['year']='Car'

df_grouped_career['OPP']='Career'
df_grouped_year['OPP']='All'

df_grouped_career['round']='Career'
df_grouped_year['round']='All'



df_final_with_career=pd.concat([df_selected,df_grouped_career,df_grouped_year])
df_final_with_career['TSA'] = df_final_with_career['FGA'] + 0.44 * df_final_with_career['FTA']


# Group by nba_id and year to roll up stats to the year

# Calculate derived statistics using safe division

zero_df = df_final_with_career[df_final_with_career.POSS==0]
df_final_with_career =  df_final_with_career[df_final_with_career.POSS!=0]
df_final_with_career['MPG'] = df_final_with_career['MIN'].divide(df_final_with_career['GP'])
df_final_with_career['PPG'] = df_final_with_career['PTS'].divide(df_final_with_career['GP'])
df_final_with_career['PTS/75'] = 75* df_final_with_career['PTS'].divide(df_final_with_career['POSS']) 
df_final_with_career['AST/75'] = 75* df_final_with_career['AST'].divide(df_final_with_career['POSS']) 
df_final_with_career['REB/75'] = 75* df_final_with_career['REB'].divide(df_final_with_career['POSS']) 
df_final_with_career['TOV/75'] = 75*df_final_with_career['TOV'].divide(df_final_with_career['POSS']) 
df_final_with_career['TSA/100'] = 100* (df_final_with_career['TSA']).divide(df_final_with_career['POSS'])
df_final_with_career['TS_ADD'] = df_final_with_career['TSA'] * df_final_with_career['rOPP_TS_PCT'] * 2
df_final_with_career['TS_ADD_1'] = df_final_with_career['TSA'] * (df_final_with_career['rOPP_TS_PCT'] + 0.01) * 2
df_final_with_career['TS_ADD_PER_100'] = 100*(df_final_with_career['TS_ADD'] ).divide(df_final_with_career['POSS'])
df_final_with_career['TS_ADD_1_PER_100'] = 100 * (df_final_with_career['TS_ADD_1']) / df_final_with_career['POSS']

extra_col = [col for col in df_final_with_career.columns if col not in zero_df]

for col in extra_col:
    zero_df[col]=0

df_final_with_career=pd.concat([df_final_with_career,zero_df])
# ... rest of the code remains the same

# Separate the rows for 'Career'
df_career = df_final_with_career[df_final_with_career['round'] == 'Career']

# Separate the rows for 'All'
df_all = df_final_with_career[df_final_with_career['round'] == 'All']

# Get the other rounds
df_rounds = df_final_with_career[(df_final_with_career['round'] != 'All')
                                 & (df_final_with_career['round'] != 'Career')]

# Create a dictionary to hold the nested structure
nested_data = []

for index, row in df_all.iterrows():
    nba_id = row['nba_id']
    year = row['year']

    # Get the rounds for this player and year
    rounds = df_rounds[(df_rounds['nba_id'] == nba_id)& (df_rounds['year'] == year)]

    # Convert the rounds to a list of dictionaries
    rounds_list = rounds.to_dict(orient='records')

    # Add the rounds to the 'All' row
    row['rounds'] = json.dumps(rounds_list)

    # Append to the nested data list
    nested_data.append(row)

# Convert the nested data list to a DataFrame
df_nested = pd.DataFrame(nested_data)

# Concatenate with the career rows
df_final_nested = pd.concat([df_nested, df_career], ignore_index=True)
df_final_nested=df_final_nested[df_final_nested['OPP_TS_PCT']>0]
# Save to CSV
output_path = 'playoffRTS.csv'
df_final_nested.to_csv(output_path, index=False)
for year in range(1997,2025):
    print(year)
    
    yrs_df = df_final_nested[df_final_nested.year==year]
    #print('Year Min TS')
    #print(yrs_df['TS_PCT'].min())
    print('Year Min OPP TS')
    print(yrs_df['OPP_TS_PCT'].min())
    #print('Year Max TS')
    #print(yrs_df['TS_PCT'].max())
    print('Year max OPP TS')
    print(yrs_df['OPP_TS_PCT'].max())

Index(['PLAYER_ID', 'PLAYER_NAME', 'NICKNAME', 'TEAM_ID', 'TEAM_ABBREVIATION',
       'AGE', 'GP', 'W', 'L', 'W_PCT',
       ...
       'REB_PCT_RANK', 'year', 'round', 'TEAM', 'OPP', 'OPP_ID', 'OPP_TS_PCT',
       'year_avg_ts', 'rOPP_TS_PCT', 'rTS_PCT'],
      dtype='object', length=131)
5665    146
5834    135
6047    161
6464    135
6604    147
6692    156
6729    156
6823     30
7004     69
7070    195
7124    158
7200    119
7365     98
7475    126
7515    134
7796     98
7871    175
7898    110
7988    148
8158    143
8247    146
8288    183
9184    140
9358    156
9454    119
9509    187
9586    236
9760    160
Name: PTS, dtype: int64
['TEAM_ID', 'year_avg_ts', 'GP', 'PTS', 'MIN', 'AGE', 'POSS', 'OFF_RATING', 'FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD', 'PLUS_MINUS']
['USG_PCT', 'TS_PCT', 'OPP_TS_PCT', 'rOPP_TS_PCT', 'AST_PCT', 'FG_PCT', 'FG3_PCT']
1997
Year Min OPP TS
50.935941018056596
Year max OPP TS
54.5

In [4]:
for year in range(1997,2025):
    print(year)
    
    yrs_df = df_selected[df_selected.year==year]
    #print('Year Min TS')
    #print(yrs_df['TS_PCT'].min())
    print('Year Min OPP TS')
    print(yrs_df['OPP_TS_PCT'].min())
    #print('Year Max TS')
    #print(yrs_df['TS_PCT'].max())
    print('Year max OPP TS')
    print(yrs_df['OPP_TS_PCT'].max())

1997
Year Min OPP TS
50.9359410180566
Year max OPP TS
54.665523909985936
1998
Year Min OPP TS
48.24676174277116
Year max OPP TS
54.51211380306735
1999
Year Min OPP TS
46.37538326763031
Year max OPP TS
52.13455520786767
2000
Year Min OPP TS
49.01440163133522
Year max OPP TS
54.364047243423016
2001
Year Min OPP TS
48.28912292968579
Year max OPP TS
53.135207250310536
2002
Year Min OPP TS
49.48383710248896
Year max OPP TS
53.93357833129168
2003
Year Min OPP TS
48.86260983821959
Year max OPP TS
53.76169438669438
2004
Year Min OPP TS
48.00683371298405
Year max OPP TS
53.7795040334628
2005
Year Min OPP TS
50.24325921884236
Year max OPP TS
54.15041033230189
2006
Year Min OPP TS
50.14003071641522
Year max OPP TS
55.003785685654485
2007
Year Min OPP TS
51.06783349959191
Year max OPP TS
56.25992191187197
2008
Year Min OPP TS
50.743502625424696
Year max OPP TS
55.24861878453039
2009
Year Min OPP TS
50.85110967463908
Year max OPP TS
55.216268277734
2010
Year Min OPP TS
51.786723470851626
Year max O

In [5]:
df_final_nested

,nba_id,player_name,year,round,OPP,USG_PCT,TS_PCT,TEAM_ID,year_avg_ts,OPP_TS_PCT,...,PTS/75,AST/75,REB/75,TOV/75,TSA/100,TS_ADD,TS_ADD_1,TS_ADD_PER_100,TS_ADD_1_PER_100,rounds
0,2,Byron Scott,1997,All,All,0.16594444444444448,58.6,3221225494,107.154334,52.819041081931445,...,15.178571,3.273810,3.571429,2.380952,16.412698,478.200922,479.028122,189.762271,190.090524,"[{""nba_id"": 2, ""player_name"": ""Byron Scott"", ""..."
1,3,Grant Long,1997,All,All,0.135,54.7,1610612765,53.577167,50.98316529038733,...,12.096774,1.451613,5.322581,0.000000,14.735484,169.78501,170.24181,109.538716,109.833426,"[{""nba_id"": 3, ""player_name"": ""Grant Long"", ""y..."
2,3,Grant Long,1999,All,All,0.17647770700636944,47.23710191082803,3221225474,102.228624,50.15738079417528,...,12.539809,0.955414,8.837580,2.746815,17.095541,-627.042282,-624.895082,-99.847497,-99.505586,"[{""nba_id"": 3, ""player_name"": ""Grant Long"", ""y..."
3,3,Grant Long,2003,All,All,0.06743076923076922,18.955384615384617,3221225476,103.803455,50.52134213603061,...,2.307692,1.153846,5.769231,1.153846,6.153846,-252.52766,-252.44766,-388.504093,-388.381016,"[{""nba_id"": 3, ""player_name"": ""Grant Long"", ""y..."
4,7,Dan Schayes,1997,All,All,0.156,44.4,1610612753,53.577167,51.19943124483992,...,9.593023,2.180233,5.232558,1.744186,14.395349,-336.707835,-336.212635,-195.760369,-195.472462,"[{""nba_id"": 7, ""player_name"": ""Dan Schayes"", ""..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7188,1641753,Chris Livingston,Car,Career,Career,0.154,43.1,1610612749,58.035355,59.36388779138681,...,12.500000,0.000000,0.000000,0.000000,19.333333,-75.464439,-75.418039,-628.870328,-628.483661,NaN
7189,1641765,Olivier-Maxence Prosper,Car,Career,Career,0.11105882352941177,0.0,3221225484,116.070710,55.161931078696604,...,0.000000,4.411765,13.235294,0.000000,11.764706,-220.647724,-220.607724,-1297.92779,-1297.692496,NaN
7190,1641767,Ben Sheppard,Car,Career,Career,0.11010846953937592,58.03789004457652,4831838262,174.106064,56.73628271378458,...,9.806835,1.894502,5.683507,1.225854,11.138187,195.136971,196.636171,28.995092,29.217856,NaN
7191,1641775,Jordan Walsh,Car,Career,Career,0.1238695652173913,47.82608695652174,3221225476,116.070710,57.42570440138734,...,6.521739,0.000000,6.521739,0.000000,13.043478,-57.597705,-57.537705,-250.424803,-250.163933,NaN


In [6]:
df_final_nested['rounds']

0       [{"nba_id": 2, "player_name": "Byron Scott", "...
1       [{"nba_id": 3, "player_name": "Grant Long", "y...
2       [{"nba_id": 3, "player_name": "Grant Long", "y...
3       [{"nba_id": 3, "player_name": "Grant Long", "y...
4       [{"nba_id": 7, "player_name": "Dan Schayes", "...
                              ...                        
7188                                                  NaN
7189                                                  NaN
7190                                                  NaN
7191                                                  NaN
7192                                                  NaN
Name: rounds, Length: 7186, dtype: object

In [8]:
df_final_with_career[df_final_with_career.player_name.str.contains('Shaq')]['round']

,nba_id,player_name,year,round,OPP,USG_PCT,TS_PCT,TEAM_ID,year_avg_ts,OPP_TS_PCT,...,PPG,PTS/75,AST/75,REB/75,TOV/75,TSA/100,TS_ADD,TS_ADD_1,TS_ADD_PER_100,TS_ADD_1_PER_100
124,406,Shaquille O'Neal,1997,1,POR,0.37,57.9,1610612747,53.577167,52.204943,...,33.000000,35.106383,3.457447,9.840426,1.861702,40.411348,1298.017403,1300.296603,460.289859,461.098086
253,406,Shaquille O'Neal,1997,2,UTA,0.341,52.3,1610612747,53.577167,53.368497,...,22.000000,27.138158,3.947368,14.309211,3.700658,34.618421,-224.897313,-222.792513,-73.979379,-73.287011
529,406,Shaquille O'Neal,1998,1,POR,0.316,62.0,1610612747,52.334623,50.709607,...,29.000000,28.618421,3.453947,11.595395,3.947368,30.750000,2110.851878,2112.721478,694.35917,694.97417
624,406,Shaquille O'Neal,1998,2,SEA,0.333,64.1,1610612747,52.334623,51.925814,...,30.600000,34.984756,4.573171,10.975610,2.972561,36.365854,2904.273833,2906.659433,885.449339,886.176656
672,406,Shaquille O'Neal,1998,3,UTA,0.377,55.1,1610612747,52.334623,52.342087,...,31.750000,32.397959,1.020408,9.438776,3.571429,39.183673,635.423251,637.727251,216.130357,216.914031
883,406,Shaquille O'Neal,1999,1,HOU,0.34,52.2,1610612747,51.114312,49.930456,...,29.500000,28.733766,3.896104,9.983766,2.191558,36.714286,513.280083,515.541683,166.649378,167.383663
973,406,Shaquille O'Neal,1999,2,SAS,0.287,50.6,1610612747,51.114312,46.375383,...,23.750000,23.360656,0.491803,12.786885,2.213115,30.767213,792.876068,794.752868,259.959367,260.574711
1052,406,Shaquille O'Neal,2000,1,SAC,0.309,53.4,1610612747,52.322075,51.74933,...,29.400000,27.840909,2.651515,16.477273,2.083333,34.737374,454.132247,456.883447,114.67986,115.374608
1229,406,Shaquille O'Neal,2000,2,PHX,0.327,55.8,1610612747,52.322075,50.302783,...,30.200000,30.280749,2.606952,16.243316,2.606952,36.149733,1486.44747,1489.15147,397.445847,398.168842
1318,406,Shaquille O'Neal,2000,3,POR,0.278,55.0,1610612747,52.322075,50.194158,...,25.857143,23.008475,3.813559,11.059322,2.415254,27.891525,1581.698844,1584.990044,268.08455,268.64238
